In [60]:
from bs4 import BeautifulSoup as bs
import requests
url = 'https://fmdataba.com/19/p/1165/lionel-messi/'

In [80]:
import pandas as pd

r = requests.get('https://fmdataba.com/19/p/1165/lionel-messi/', headers = {'User-Agent':'Mozilla/5.0'})
soup = bs(r.content, 'html.parser')
abilities = ['TECHNICAL', 'MENTAL' , 'PHYSICAL']

def get_abilities(soup, keyword):
    table = soup.select_one('div:has(h3:contains("' + ability + '")) ~ div > table')
    d = {item.select_one('td:nth-child(odd)').text:item.select_one('td:nth-child(even)').text for item in table.select('tr')}
    return d

results = {}

for ability in abilities:
    results[ability] = get_abilities(soup, ability)
    
# Get Teammates URLs

table = soup.select_one('div:has(h3:contains("' + 'Teammates' + '")) ~ div > table')
# print('table', table.find_all('a')['href'])
for a in table.find_all('a', href=True):
    print("Found the URL:", a['href'])

Found the URL: https://fmdataba.com/19/p/1170/luis-suarez/
Found the URL: https://fmdataba.com/19/p/1154/philippe-coutinho/
Found the URL: https://fmdataba.com/19/p/1156/ousmane-dembele/
Found the URL: https://fmdataba.com/19/p/1134/sergio-busquets/
Found the URL: https://fmdataba.com/19/p/1112/gerard-pique/
Found the URL: https://fmdataba.com/19/p/1140/ivan-rakitic/
Found the URL: https://fmdataba.com/19/p/1124/sergi-roberto/
